In [9]:
#%matplotlib inline
%time from hikyuu.interactive import *

CPU times: user 119 μs, sys: 477 μs, total: 596 μs
Wall time: 623 μs


# 1 利用 TM 实现简单的记账本

TradeManager对象可以理解为一个模拟的交易账户，负责交易的买/卖操作、记录交易记录以及持仓情况，也可以通过修改其买/卖操作的接口实现实盘接入。创建一个模拟交易账户，通常使用快捷创建函数 crtTM。TM对象的基本操作：

- buy  买入
- sell 卖出
- checkin 存入现金
- checkout 取出现金

可以利用 TM 实现简单的记账本，手工记录自己的操作情况，例如：

In [10]:
#创建一个初始资金10万元，起始日期2017年1月1日的模拟账户
my_tm = crtTM(init_cash=100000, date=Datetime(201701010000))

#2017年1月3日以9.11的价格买入100股
td = my_tm.buy(Datetime(201701030000), sm['sz000001'], 9.11, 100)

#查看当前资金及持仓情况
print(my_tm)

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-01-03 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current total funds: 100005.00,
  current cash: 99089.00,
  current market_value: 916.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
    SZ000001 平安银行 2017-01-03 00:00:00 2200 100.00 911.00 1106.90 195.90 21.50% 0.20%
  Short Position: 
  Borrow Stock: 
}


In [11]:
#转化为pandas的DataFrame显示当前持仓情况 
position = my_tm.get_position_list()
position.to_df()

,证券代码,证券名称,开仓时间,持有天数,持仓数量,总投入资金,市值,收益,收益百分比,止损,目标价格,清仓时间,累计持仓数量,累计成本,累计风险,累计买入资金,累计卖出资金
0,SZ000001,平安银行,2017-01-03,3307,100.0,911.0,1106.9,195.9,21.5,0.0,0.0,NaT,100.0,0.0,911.0,911.0,0.0


In [12]:
my_tm.get_trade_list().to_df()

,证券代码,证券名称,日期,交易操作,计划价格,实际价格,目标价格,数量,止损,现金,总成本,佣金,印花税,过户费,其它成本,来源,备注
0,,,2017-01-01,INIT,100000.0,100000.00,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,0.0,0.0,--,
1,SZ000001,平安银行,2017-01-03,BUY,0.0,9.11,0.0,100.0,0.0,99089.0,0.0,0.0,0.0,0.0,0.0,--,


In [13]:
#2017年2月21日以9.60的价格卖出100股
td = my_tm.sell(Datetime(201702210000), sm['sz000001'], 9.60)

my_tm

TradeManager {
  params: params[precision(int): 2, save_action(bool): 1, support_borrow_cash(bool): 0, support_borrow_stock(bool): 0, ],
  name: SYS,
  init_date: 2017-01-01 00:00:00,
  init_cash: 100000.00,
  firstDatetime: 2017-01-03 00:00:00,
  lastDatetime: 2017-02-21 00:00:00,
  TradeCostFunc: TradeCostFunc(TC_Zero, params[]),
  current total funds: 100049.00,
  current cash: 100049.00,
  current market_value: 0.00,
  current short_market_value: 0.00,
  current base_cash: 100000.00,
  current base_asset: 0.00,
  current borrow_cash: 0.00,
  current borrow_asset: 0.00,
  Position: 
  Short Position: 
  Borrow Stock: 
}

# 2 利用 Excel 查看交易详情

使用 tocsv 方法将 TM 的交易记录、当前持仓及已平仓详细情况分别保存为 csv 文件，以便用 Excel 查看详情。

tocsv方法参数为一个指定的目录，目录必须以存在。其输出会在指定目录中，生成三个文件，“TM名称_交易记录.csv”、“TM名称_未平仓记录.csv”、“TM名称_已平仓记录.csv”。TM名称可在crtTM创建TM对象时指定，默认为“SYS”，如下图所示。

<img src="images/008_01_tocsv.png" align='left'>

In [14]:
#在 hikyuu_XXX.ini 文件中配置的临时路径中输出
my_tm.tocsv(sm.tmpdir())

2026-01-23 01:29:42.637 [HKU-E] - Can't create file d:/stock/SYS_交易记录.csv! (TradeManager.cpp:1640)


使用 Excel 查看 csv，如：

<img src="images/008_02_tocsv_look.png" align="left">

# 3 使用序列化保存或重新载入已有TM对象

In [ ]:
#保存至指定文件
from datetime import date
filename = "my_trade_record_{}.pkl".format(date.today())
hku_save(my_tm, filename)

In [ ]:
#载入已保存的TM对象
new_my_tm = hku_load(filename)

# 4 使用订单代理

In [17]:
#创建模拟交易账户进行回测，初始资金30万
my_tm = crtTM(init_cash=300000, date=Datetime(201701010000))

#注册实盘交易订单代理
ob = crtOB(TestOrderBroker())
my_tm.reg_broker(ob) #TestOerderBroker是测试用订单代理对象，只打印
# 注意：pybind 不支持下面这种方式调用，必须先生成实例再传入！！！
# my_tm.reg_broker(crtOB(TestOrderBroker(), False))

#根据需要修改订单代理最后的时间戳，后续只有大于该时间戳时，订单代理才会实际发出订单指令
my_tm.broker_last_datetime=Datetime(201701010000)

#创建信号指示器（以5日EMA为快线，5日EMA自身的10日EMA作为慢线，快线向上穿越慢线时买入，反之卖出）
my_sg = SG_Flex(EMA(C, n=5), slow_n=10)

#固定每次买入1000股
my_mm = MM_FixedCount(1000)

#创建交易系统并运行
sys = SYS_Simple(tm = my_tm, sg = my_sg, mm = my_mm)
sys.run(sm['sz000001'], Query(-150))

买入：SZ000001, 价格: 11.81, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 12.46, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 12.48, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 12.4, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 12.450000000000001, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 12.09, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.5, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.41, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.52, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.38, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.59, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信号来源: SystemPart.SIGNAL, 备注: 
卖出：SZ000001, 价格: 11.6, 数量: 1000.0, 信号来源: SystemPart.SIGNAL, 备注: 
买入：SZ000001, 价格: 11.63, 数量: 1000.0 预期止损价: 0.0, 预期目标价: nan, 信

In [18]:
my_tm.get_trade_list().to_df()

,证券代码,证券名称,日期,交易操作,计划价格,实际价格,目标价格,数量,止损,现金,总成本,佣金,印花税,过户费,其它成本,来源,备注
0,,,2017-01-01,INIT,300000.000,300000.000,0.0,0.0,0.0,300000.0,0.0,0.0,0.0,0.0,0.0,--,
1,SZ000001,平安银行,2025-06-23,BUY,11.810,11.810,NaN,1000.0,0.0,288190.0,0.0,0.0,0.0,0.0,0.0,SG,
2,SZ000001,平安银行,2025-07-23,SELL,12.460,12.460,NaN,1000.0,0.0,300650.0,0.0,0.0,0.0,0.0,0.0,SG,
3,SZ000001,平安银行,2025-08-08,BUY,12.480,12.480,NaN,1000.0,0.0,288170.0,0.0,0.0,0.0,0.0,0.0,SG,
4,SZ000001,平安银行,2025-08-11,SELL,12.400,12.400,NaN,1000.0,0.0,300570.0,0.0,0.0,0.0,0.0,0.0,SG,
5,SZ000001,平安银行,2025-08-26,BUY,12.450,12.450,NaN,1000.0,0.0,288120.0,0.0,0.0,0.0,0.0,0.0,SG,
6,SZ000001,平安银行,2025-08-28,SELL,12.090,12.090,NaN,1000.0,0.0,300210.0,0.0,0.0,0.0,0.0,0.0,SG,
7,SZ000001,平安银行,2025-10-17,BUY,11.500,11.500,NaN,1000.0,0.0,288710.0,0.0,0.0,0.0,0.0,0.0,SG,
8,SZ000001,平安银行,2025-10-20,SELL,11.410,11.410,NaN,1000.0,0.0,300120.0,0.0,0.0,0.0,0.0,0.0,SG,
9,SZ000001,平安银行,2025-10-23,BUY,11.520,11.520,NaN,1000.0,0.0,288600.0,0.0,0.0,0.0,0.0,0.0,SG,


In [19]:
my_tm.get_trade_list().to_np()

array([('', '', '2017-01-01T00:00:00.000000000', 'INIT', 3.0000e+05, 3.0000e+05,  0.,    0., 0., 300000., 0., 0., 0., 0., 0., '--', ''),
       ('SZ000001', '平安银行', '2025-06-23T00:00:00.000000000', 'BUY', 1.1810e+01, 1.1810e+01, nan, 1000., 0., 288190., 0., 0., 0., 0., 0., 'SG', ''),
       ('SZ000001', '平安银行', '2025-07-23T00:00:00.000000000', 'SELL', 1.2460e+01, 1.2460e+01, nan, 1000., 0., 300650., 0., 0., 0., 0., 0., 'SG', ''),
       ('SZ000001', '平安银行', '2025-08-08T00:00:00.000000000', 'BUY', 1.2480e+01, 1.2480e+01, nan, 1000., 0., 288170., 0., 0., 0., 0., 0., 'SG', ''),
       ('SZ000001', '平安银行', '2025-08-11T00:00:00.000000000', 'SELL', 1.2400e+01, 1.2400e+01, nan, 1000., 0., 300570., 0., 0., 0., 0., 0., 'SG', ''),
       ('SZ000001', '平安银行', '2025-08-26T00:00:00.000000000', 'BUY', 1.2450e+01, 1.2450e+01, nan, 1000., 0., 288120., 0., 0., 0., 0., 0., 'SG', ''),
       ('SZ000001', '平安银行', '2025-08-28T00:00:00.000000000', 'SELL', 1.2090e+01, 1.2090e+01, nan, 1000., 0., 300210., 0.,

In [20]:
my_tm.get_history_position_list().to_df()

,证券代码,证券名称,开仓时间,持有天数,持仓数量,总投入资金,市值,收益,收益百分比,止损,目标价格,清仓时间,累计持仓数量,累计成本,累计风险,累计买入资金,累计卖出资金
0,SZ000001,平安银行,2025-06-23,30,0.0,-650.0,0.0,650.0,-100.0,0.0,NaN,2025-07-23,1000.0,0.0,11810.0,11810.0,12460.0
1,SZ000001,平安银行,2025-08-08,3,0.0,80.0,0.0,-80.0,-100.0,0.0,NaN,2025-08-11,1000.0,0.0,12480.0,12480.0,12400.0
2,SZ000001,平安银行,2025-08-26,2,0.0,360.0,0.0,-360.0,-100.0,0.0,NaN,2025-08-28,1000.0,0.0,12450.0,12450.0,12090.0
3,SZ000001,平安银行,2025-10-17,3,0.0,90.0,0.0,-90.0,-100.0,0.0,NaN,2025-10-20,1000.0,0.0,11500.0,11500.0,11410.0
4,SZ000001,平安银行,2025-10-23,7,0.0,140.0,0.0,-140.0,-100.0,0.0,NaN,2025-10-30,1000.0,0.0,11520.0,11520.0,11380.0
5,SZ000001,平安银行,2025-11-05,26,0.0,-10.0,0.0,10.0,-100.0,0.0,NaN,2025-12-01,1000.0,0.0,11590.0,11590.0,11600.0
6,SZ000001,平安银行,2025-12-19,12,0.0,151.0,0.0,-151.0,-100.0,0.0,NaN,2025-12-31,1000.0,0.0,11630.0,11630.0,11479.0
7,SZ000001,平安银行,2026-01-07,5,0.0,210.0,0.0,-210.0,-100.0,0.0,NaN,2026-01-12,1000.0,0.0,11659.0,11659.0,11449.0


In [21]:
my_tm.get_history_position_list().to_np()

array([('SZ000001', '平安银行', '2025-06-23T00:00:00.000000000', 30, 0., -650., 0.,  650., -100., 0., nan, '2025-07-23T00:00:00.000000000', 1000., 0., 11810., 11810., 12460.),
       ('SZ000001', '平安银行', '2025-08-08T00:00:00.000000000',  3, 0.,   80., 0.,  -80., -100., 0., nan, '2025-08-11T00:00:00.000000000', 1000., 0., 12480., 12480., 12400.),
       ('SZ000001', '平安银行', '2025-08-26T00:00:00.000000000',  2, 0.,  360., 0., -360., -100., 0., nan, '2025-08-28T00:00:00.000000000', 1000., 0., 12450., 12450., 12090.),
       ('SZ000001', '平安银行', '2025-10-17T00:00:00.000000000',  3, 0.,   90., 0.,  -90., -100., 0., nan, '2025-10-20T00:00:00.000000000', 1000., 0., 11500., 11500., 11410.),
       ('SZ000001', '平安银行', '2025-10-23T00:00:00.000000000',  7, 0.,  140., 0., -140., -100., 0., nan, '2025-10-30T00:00:00.000000000', 1000., 0., 11520., 11520., 11380.),
       ('SZ000001', '平安银行', '2025-11-05T00:00:00.000000000', 26, 0.,  -10., 0.,   10., -100., 0., nan, '2025-12-01T00:00:00.000000000', 1000